In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
import sys
sys.path.append('/home/cizinsky/garment-texture-completion')
CKPT_ROOT = '/scratch/izar/cizinsky/garment-completion/checkpoints'

from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider
import pytorch_lightning as pl
from pytorch_lightning import Trainer


import torch
from torchvision.transforms.functional import pil_to_tensor
from torchvision.utils import make_grid

from helpers.pl_module import GarmentInpainterModule
from helpers.dataset import get_dataloaders
from helpers.data_utils import denormalise_image_torch
from helpers.data_utils import torch_image_to_pil, denormalise_image_torch
from helpers.metrics import compute_all_metrics
from helpers.utils import get_model_data, get_model



from tqdm import tqdm
import wandb
import pandas as pd

### Ablations

In [3]:
run_name = "twilight-sponge-160"
trn_dataloader, val_dataloader = get_model_data(run_name, debug_val_size=2)

In [4]:
model, cfg = get_model(run_name, trn_dataloader)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
trainer = Trainer(accelerator="gpu", devices=1, logger=False, callbacks=[])

/home/cizinsky/venvs/garment/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /home/cizinsky/venvs/garment/lib/python3.10/site-pa ...
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
# Predict
outputs = trainer.predict(model, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Using text guidance scale: 1.5 and image guidance scale: 5.0


  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
all_ssim = torch.cat([output["ssim"] for output in outputs])
all_psnr = torch.cat([output["psnr"] for output in outputs])
all_lpips = torch.cat([output["lpips"] for output in outputs])

mean_ssim = all_ssim.mean()
mean_psnr = all_psnr.mean()
mean_lpips = all_lpips.mean()

print(f"Mean SSIM: {mean_ssim}, Mean PSNR: {mean_psnr}, Mean LPIPS: {mean_lpips}")

Mean SSIM: 0.15332093834877014, Mean PSNR: 16.306791305541992, Mean LPIPS: 0.6907849311828613


In [15]:
run = wandb.init(project="pbr-generation", entity="ludekcizinsky", resume="must", id="fstkreeo")

In [16]:
# your post‐train metrics
final_metrics = {
    "dev/ssim": mean_ssim,
    "dev/psnr": mean_psnr,
    "dev/lpips": mean_lpips
}

# write them to the summary
run.summary.update(final_metrics)

run.finish()

dev/lpips,0.69078
dev/psnr,16.30679
dev/ssim,0.15332
epoch,0
optim/grad_norm_postclip,0.35911
optim/grad_norm_preclip,0.35911
optim/lr,1e-05
train/ddim_loss,0.19605
train/loss,0.1647
train/mse_loss,0.11569
trainer/global_step,22489
